<a href="https://colab.research.google.com/github/kachytronico/colab-PIA/blob/main/204_An%C3%A1lisis_de_variabilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de variabilidad

Durante los cuadernillos anteriores hemos visto cómo eliminar datos inútiles o nulos (o crear datos sintéticos para usarlos más adelante).

Sin embargo, existen otros datos que podría interesarnos eliminar para depurar nuestro conjunto de datos: los datos con una **variabilidad baja**.

Estos datos, básicamente, aumentan las dimensiones de nuestro conjunto de datos para no incluir información relevante o útil.

Imagínate un conjunto de datos que contenga una columna ```edad``` y que, de 1000 datos, 999 tengan 19 años y 1 tenga 20. Esa columna ```edad``` tiene una variabilidad tan baja que no tiene ningún sentido su existencia.

Es importante conocer que los datos nulos no explican una variable. Por ejemplo, si de una columna con 1000 datos tuviésemos 995 datos nulos, la variabilidad de la columna sería de 5/1000 (0.5%), y posiblemente sería descartada.

Para ver cómo vamos a trabajar con este tipo de datos, vamos a utilizar el conjunto de datos del Titanic tal y como lo hemos dejado anteriormente en 202 y 203.

## Creando el ```dataset``` del Titanic: carga de datos

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

df = sns.load_dataset("titanic")

## Creando el ```dataset``` del Titanic: eliminación de columnas inútiles

In [ ]:
to_remove_columns = ["class", "who", "adult_male", "embarked", "alive"]
df = df.drop(columns = to_remove_columns)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,deck,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,NaN,Southampton,False
1,1,1,female,38.0,1,0,71.2833,C,Cherbourg,False
2,1,3,female,26.0,0,0,7.9250,NaN,Southampton,True
3,1,1,female,35.0,1,0,53.1000,C,Southampton,False
4,0,3,male,35.0,0,0,8.0500,NaN,Southampton,True


## Creando el ```dataset``` del Titanic: tratamiento de nulos (eliminación e imputación)

Variable ```emabark_town``` con 2 valores nulos: eliminación.

In [ ]:
df = df[df.embark_town.notnull()]

Variable ```age``` con 200 nulos: imputación.

In [ ]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy = "mean")
pred = si.fit_transform(df.age.values.reshape(-1, 1)) # recuerda: formato de 1 columna rellenada con listas de un valor
df.age = pred
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,deck,embark_town,alone
0,0,3,male,22.0,1,0,7.2500,NaN,Southampton,False
1,1,1,female,38.0,1,0,71.2833,C,Cherbourg,False
2,1,3,female,26.0,0,0,7.9250,NaN,Southampton,True
3,1,1,female,35.0,1,0,53.1000,C,Southampton,False
4,0,3,male,35.0,0,0,8.0500,NaN,Southampton,True


## Situación inicial del cuadenillo 204

Esta es nuestra situación inicial.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     889 non-null    int64   
 1   pclass       889 non-null    int64   
 2   sex          889 non-null    object  
 3   age          889 non-null    float64 
 4   sibsp        889 non-null    int64   
 5   parch        889 non-null    int64   
 6   fare         889 non-null    float64 
 7   deck         201 non-null    category
 8   embark_town  889 non-null    object  
 9   alone        889 non-null    bool    
dtypes: bool(1), category(1), float64(2), int64(4), object(2)
memory usage: 64.6+ KB


En particular, vamos a ver qué hacemos con la variable ```deck``` y qué hacemos con las demás, que pueden necesitar también un análisis de variabilidad.

## Usando la variablidad con columnas de datos finitos (categóricos)

Vamos a comenzar analizando las variables categóricas. Recuerda que estas variables toman valores dentro de un rango finito de opciones. Son ejemplos de variables categóricas: ```survived``` (por ser booleana) o ```pclass``` (por ser una categoría).

En estos casos, no nos interesa conocer cuántos valores distintos hay (porque son finitos, así que generalmente habrá muchísimos repetidos), sino cuántos hay de cada uno de ellos.

Si hubiera un gran desbalanceo entre unas clases y otras (se denomina **conjunto de datos desbalanceado** a los conjuntos de datos que tienen estos problemas), sería interesante plantearse si borrar o no la columna.

Para contar las apariciones de distintos valores, usaremos la función ```value_counts```.

In [ ]:
df.survived.value_counts()

,count
survived,
0,549
1,340


En este caso, existe una gran diferencia que posiblemente esté por encima del 20%:

In [ ]:
(549 - 340) * 100.0 / 889 # diferencia entre el total de datos

23.509561304836897

Este caso es muy frecuente en conjuntos de datos de desastres naturales o tratamiento médico: lo normal es que los datos pertenezcan a una sola clase (muertos u organismos enfermos/sanos).

En este caso, supera un poco el límite del 20%, pero no podemos eliminar esta variable por dos razones:
- Solo tiene dos valores, por lo que no podemos ser tan estrictos con el límite (posilemente, estaría entre el 60% y el 70%).
- Es una variable fundamental del conjunto de datos, dado que mide la cantidad de gente que murió.

Veamos otra variable:

In [ ]:
df.pclass.value_counts()

,count
pclass,
3,491
1,214
2,184


Por ejemplo, pudiera sorprendernos que hubiese más personas viajando en primera clase que en segunda.

Esta variable también está desbalanceada, dado que la clase más representada tiene más del doble que las otras clases, pero no que su suma.

Ten en cuenta que, si tenemos más de dos variables, la variabilidad esa variable queda explicada de forma comparativa entre todas las clases:

In [ ]:
(491 - (214 + 184)) * 100.0 / 889

10.46119235095613

En este caso, la variabilidad está en el 10.5%.

In [ ]:
df.sibsp.value_counts() # esta variable la habíamos apartado desde el principio

,count
sibsp,
0,606
1,209
2,28
4,18
3,16
8,7
5,5


In [ ]:
values = df.sibsp.value_counts()
(values[0] - sum(values[1:])) * 100.0 / sum(values) # quédate con esta línea, código ÚTIL

np.float64(36.33295838020248)

En este caso, la diferencia respecto a la clase más representada supera el 20%, por lo que podríamos plantearnos eliminar esta variable.

In [ ]:
values = df.parch.value_counts()
(values[0] - sum(values[1:])) * 100.0 / sum(values)

np.float64(52.08098987626547)

Igual que la anterior.

En este punto, podrías plantearte... Si siempre hacemos el mismo análisis, ¿por qué no lo automatizamos abstrayéndolo a un nuevo método?

La respuesta es relativamente sencilla. Probemos con la variable ```age```.

In [ ]:
df.age.value_counts()

,count
age,
29.642093,177
24.000000,30
22.000000,27
18.000000,26
28.000000,25
...,...
24.500000,1
0.670000,1
0.420000,1


Tenemos muchos valores de edad, la mayoría con 1, pero hay otros que se repiten (jóvenes). Además, tenemos el valor que hemos imputado antes para los nulos. ¿Cuántos valores distintos tenemos?

In [ ]:
len(df.age.unique())

89

Tenemos 89 valores distintos, pero... ¿es una variable categórica o numérica?

Para determinarlo, podríamos usar un límite que, al sobrepasarse, indicase que es una variable numérica. Fíjate que, aunque la variable contenga números, puede no ser numérica (por ejemplo, la variable ```pclass```).

En este caso, es una variable categórica, pero con **muchísimas clases**. Por eso, no podemos generalizar este problema (de forma sencilla ahora, lo acabaremos haciendo más adelante), porque esta variable sería eliminada. Sería considerada una variable numérica con muy poca variabilidad.

Nos pasa lo mismo con la variable ```fare```.

In [ ]:
df.embark_town.value_counts()

,count
embark_town,
Southampton,644
Cherbourg,168
Queenstown,77


In [ ]:
df.alone.value_counts()

,count
alone,
True,535
False,354


De todas las variables categóricas, podríamos eliminar: ```sibsp```, ```parch``` y ```embark_town```.

In [ ]:
df = df.drop(columns=["sibsp", "parch", "embark_town"])
df.head()

,survived,pclass,sex,age,fare,deck,alone
0,0,3,male,22.0,7.2500,NaN,False
1,1,1,female,38.0,71.2833,C,False
2,1,3,female,26.0,7.9250,NaN,True
3,1,1,female,35.0,53.1000,C,False
4,0,3,male,35.0,8.0500,NaN,True


## Usando la variablidad con columnas de datos infinitos

En este conjunto de datos no sucede, pero podríamos tener una variable que fuese numérica y que pudiese tomar cualquier valor. Generalmente, cuanto más grande sea el conjunto de datos más raro es encontrar estas variables, porque los datos empezarán a repetirse.

De hecho, podemos observar este suceso en nuestra columna ```fare```.

In [ ]:
df.fare.value_counts()

,count
fare,
8.0500,43
13.0000,42
7.8958,38
7.7500,34
26.0000,31
...,...
13.8583,1
50.4958,1
5.0000,1


Vemos que 43 personas compraron el tiquet por 8\$, que 42 lo hicieron por 13$, etcétera.

En general, aplicando cualquier análisis, obtendríamos que estas variables numéricas contínuas tienen muy baja variabilidad (se espera que las variables numéricas contínuas no se repitan nunca o casi nunca), por lo que un análisis automático nos haría eliminar esta columna incorrectamente.

En general, y por zanjar el tema de **cuándo y cuándo NO** borrar una columna: borraremos columnas por tener poca variabilidad cuando sea **evidente** que existe un problema en dicha variabilidad. Por ejemplo, 500 valores iguales y 10 distintos.

Sin embargo, tendremos que tener especial cuidado a la hora de eliminar variables por esta razón.

## Usando la variabilidad con columnas de datos nulos

Finalmente, tenemos que tener en cuenta también las columnas con datos nulos.

Un dato nulo no aumenta la variabilidad, por lo que columnas con más de un 50% de datos nulos serán candidatas a ser eliminadas. Fíjate que, si está por debajo del 50%, puede tener sentido imputarlos, pero no por encima del 50%, dado que es el límite a partir del cuál los estadísticos se vuelven _medio locos_. Sería un caso extremo de un conjunto con muy pocos datos en el que tendríamos que imputar todos estos valores o, por otra parte, eliminar esta característica.

En el caso de la variable ```deck``` observamos que hay una extrema cantidad de nulos:

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 889 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   survived  889 non-null    int64   
 1   pclass    889 non-null    int64   
 2   sex       889 non-null    object  
 3   age       889 non-null    float64 
 4   fare      889 non-null    float64 
 5   deck      201 non-null    category
 6   alone     889 non-null    bool    
dtypes: bool(1), category(1), float64(2), int64(2), object(1)
memory usage: 43.8+ KB


In [ ]:
1 - (201 / 889)

0.7739032620922385

Por lo que está más que justificada su eliminación.

In [ ]:
df = df.drop(columns=["deck"])
df.head()

,survived,pclass,sex,age,fare,alone
0,0,3,male,22.0,7.2500,False
1,1,1,female,38.0,71.2833,False
2,1,3,female,26.0,7.9250,True
3,1,1,female,35.0,53.1000,False
4,0,3,male,35.0,8.0500,True


Como puedes ver, cada vez tenemos un conjunto de datos más pequeño (con menos dimensiones) y más limpio (con menos datos nulos).

Este es el objetivo, precisamente, del preprocesamiento de la información.

Durante el siguiente cuadernillo veremos cómo tratar las columnas categóricas, dado que **NO** podemos entrenar un modelo clásico con datos que sean texto (salvo la excepción del árbol de decisión).